# TD 2B : Python autres langages (correction)

* [Exercice 1 : régression sous R](#exo1)
* [Exercice 2 : distance d'édition en Cython](#exo2)


<h3 id="exo1">Exercice 1 : régression sous R</h3>

L'exercice consiste à simuler des données en python selon le modèle $Y= 3X_1 -2X_2 + 2 + \epsilon$ avec $X_1,X_2,\epsilon \sim \mathcal{N}(0,1)$ puis à utiliser R pour retrouver les coefficients de la régression. Vous pouvez regarder cette page [Multiple Linear Regression](http://www.statmethods.net/stats/regression.html).

<h3 id="exo2">Exercice 2 : distance d'édition en Cython</h3>

La fonction qui suit implémente la [distance de Lenvestein](http://fr.wikipedia.org/wiki/Distance_de_Levenshtein) qui est une distance entre deux mots. Il faut la transposer en Cython et mesurer le gain de temps avec [%timeit](http://nbviewer.ipython.org/gist/damontallen/5978528).

In [1]:
%load_ext cythonmagic

In [33]:
%%cython
cimport cython

#from libcpp.unordered_map cimport unordered_map  # j'ai essayé avec des objets purement C++
#from libcpp.map cimport map                      # mais la syntaxe n'est pas toujours évidente à deviner
                                                  # et il faut avoir le même compilateur que celui utilisé
                                                  # pour l'interpréteur Python

cdef inline int int_min(int a, int b): return a if a <= b else b
cdef inline double  double_min(double a, double b): return a if a <= b else b

cdef double c_distance_edition(str mot1, str mot2):
    cdef dict dist = { }
    cdef double m 
    cdef int l1 = len(mot1)
    cdef int l2 = len(mot2)
    cdef double mx = l1 + l2
    
    dist [(-1,-1)] = 0.0
    
    for i from 0 <= i < l1 :
        dist[i,-1] = dist[i-1,-1] + 1
        dist[-1,i] = dist[-1,i-1] + 1
        c = mot1[i]
        for j from 0 <= j < l2 :
            m = mx
            if dist.has_key( (i-1,j) ) :
                x = dist[i-1,j] + 1
                m = double_min(m,x)
            if dist.has_key( (i,j-1) ):
                x = dist[i,j-1] + 1
                m = double_min(m,x)
            if dist.has_key ( (i-1,j-1) ) :
                d = mot2[j]
                x = dist[i-1,j-1] + (1 if c != d else 0)
                m = double_min(m,x)
            dist[i,j] = m
    return dist[len(mot1)-1,len(mot2)-1]

# l'utilisation de cython n'est pas toujours facile
# car cela fait intervenir de nombreux outils (compilateur python)
# dans cette cellule, il faut ajouter une fonction python qui
# appelle la fonction cython
def distance_edition(mot1,mot2):
    return c_distance_edition(mot1,mot2)

In [34]:
%timeit distance_edition("example","exemple")

10000 loops, best of 3: 77.1 µs per loop


La fonction est aussi rapide que les calculs se fassent en ``double`` ou ``int`` ce qui n'est pas le cas pour la version Python. Dans le cas de la fonction Cython, les conversions implicite se font lors de la compilation et non lors de l'exécution.